In [37]:
from dotenv import load_dotenv
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # Now this import should work
import openai
import faiss 
load_dotenv()

import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ['LANGCHAIN_PROJECT'] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
os.environ['LANGCHAIN_TRACING_V2'] = "True" # 'true'를 'True'로 변경
openai.api_key = ''

In [30]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/mnews/article/022/0003944893",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {docs[0].page_content}")

문서의 수: 
엔비디아와 한국 밸류업 [더 나은 경제, SDGs]


			지난 18일(이하 현지시간) 미국 뉴욕증권거래소(NYSE)에 상장된 엔비디아의 주가가 전장보다 3.51% 올라 135.58달러에 마감했다. 이날 종가 기준으로 엔비디아의 시가총액은 3조 3350억달러(약 4609조원)로 올라서 지난 5년간 1위 자리를 서로 번갈아 지켰던 마이크로소프트(MS·3조3173억달러)와 애플(3조2859억달러)을 넘어섰다. 5년 전 엔비디아의 시총이 20위권 밖에 있었던 사실을 감안하면 그야말로 천지개벽이다.     한국 주식시장에 상장된 모든 기업의 시총은 도합 2900조원에 조금 못 미치는 상황이니 산술적으로 계산하면 한국 증시에서 모든 기업의 주식을 다 팔아도 엔비디아 전체를 살 수 없는 셈이다. 한국 증시 투자자 입장에서는 뼈아픈 지점이다. ‘주식이민’이라는 말이 빈번히 나올 수밖에 없는 상황이다.  



   미국 캘리포니아주 산타클라라에 위치한 엔비디아 본사 건물의 모습. 산타클라라=AP연합뉴스       미국에서 급성장한 기술기업이 그렇지만 엔비디아 역시 영화에서 나올법한 성장 스토리를 가지고 있다. 사무실이 없어 이곳 저것 레스토랑을 전전하던 젠슨 황과 크리스 말라초스키, 커티스 프리엠은 1993년 캘리포니아주 샌프란시스코 실리콘밸리의 작은 식당 ‘데니스’에서 엔비디아를 창업했다. 그들은 당시 유행하던 비디오 게임을 조금 더 고성능으로 구현하고자 게임용 그래픽 칩 제작을 시작했는데, 초기에 이들의 제품은 주목받지 못했지만 이러한 시도와 기술 개발이 훗날 그래픽처리장치(GPU)의 밑거름이 됐다.     2022년 미국 오픈AI사의 생성형 인공지능(AI) 제품 ‘챗GPT’가 개발되면서 엔비디아의 가치는 수직 상승했다. 생성형 AI를 개발하는데 GPU가 필수적이기 때문이다. 엔비디아는 전 세계 GPU 시장의 80%를 점유하고 있고, AI 가속기 시장은 98%를 차지하며 독점적 지위를 자랑하고 있다.    기술주들 덕분에 뉴욕 증시는 연일 최대치를 경신

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/022/0003944893'}, page_content='엔비디아와 한국 밸류업 [더 나은 경제, SDGs]\n\n\n\t\t\t지난 18일(이하 현지시간) 미국 뉴욕증권거래소(NYSE)에 상장된 엔비디아의 주가가 전장보다 3.51% 올라 135.58달러에 마감했다. 이날 종가 기준으로 엔비디아의 시가총액은 3조 3350억달러(약 4609조원)로 올라서\xa0지난 5년간 1위 자리를 서로 번갈아 지켰던 마이크로소프트(MS·3조3173억달러)와 애플(3조2859억달러)을 넘어섰다. 5년 전 엔비디아의 시총이 20위권 밖에 있었던 사실을 감안하면 그야말로 천지개벽이다.  \xa0  한국 주식시장에 상장된 모든 기업의 시총은 도합\xa02900조원에 조금 못 미치는 상황이니\xa0산술적으로 계산하면 한국 증시에서 모든 기업의 주식을 다 팔아도 엔비디아 전체를\xa0살 수 없는 셈이다. 한국 증시 투자자 입장에서는 뼈아픈 지점이다. ‘주식이민’이라는 말이 빈번히 나올 수밖에 없는 상황이다.'),
 Document(metadata={'source': 'https://n.news.naver.com/mnews/article/022/0003944893'}, page_content='미국 캘리포니아주 산타클라라에 위치한 엔비디아 본사 건물의 모습. 산타클라라=AP연합뉴스    \xa0  미국에서 급성장한 기술기업이 그렇지만 엔비디아 역시 영화에서\xa0나올법한 성장 스토리를 가지고 있다. 사무실이 없어 이곳 저것 레스토랑을 전전하던 젠슨 황과\xa0크리스 말라초스키, 커티스 프리엠은\xa01993년 캘리포니아주 샌프란시스코\xa0실리콘밸리의 작은 식당\xa0‘데니스’에서 엔비디아를 창업했다. 그들은 당시 유행하던 비디오 게임을 조금 더 고성능으로 구현하고자\xa0게임용 그래픽 칩 제작을 시작했는데, 초기에 이들의 제품은\xa0주목받지\xa0못했지만 이러

In [38]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
print(vectorstore)
print(retriever)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)